In [ ]:
import os
import random
import matplotlib.pyplot as plt
import seaborn as sns
from os.path import join
from tqdm import tqdm
import pandas as pd
from typing import List
import numpy as np
import sasc.viz
from pprint import pprint
import pandas as pd
import joblib
from collections import defaultdict
from sasc.config import RESULTS_DIR, REPO_DIR, CACHE_DIR
import sys
import json

sys.path.append(join(REPO_DIR, "notebooks_stories", "0_voxel_select"))
subject = 'UTS01'

In [36]:
# 8 voxels, 2 explanations each, 2 stories

r = pd.read_pickle(join(RESULTS_DIR, "fmri_results_merged.pkl")).sort_values(
    by=["stability_score"], ascending=False
)
r["id"] = (
    "('"
    + r["top_explanation_init_strs"].str.replace(" ", "_").str.slice(stop=20)
    + "', '"
    + r["subject"]
    + "', "
    + r["module_num"].astype(str)
    + "),"
)
r["id_tup"] = r.apply(
    lambda row: (
        row["top_explanation_init_strs"].replace(" ", "_")[:20],
        row["subject"],
        row["module_num"],
    ),
    axis=1,
)
r = r[r.subject == subject]


with pd.option_context(
    "display.max_rows", None, "display.max_columns", None, "display.max_colwidth", 400
):
    display(
        r[
            [
                "id",
                "top_explanation_init_strs",
                "top_explanation_init_strs_llama",
                "explanation_init_strs",
                "stability_score",
                # "fmri_test_corr",
                # "frac_top_ngrams_module_correct",
                # "top_score_normalized",
                # "top_ngrams_module_correct",
            ]
        ]
        .sort_values(by="stability_score", ascending=False)
        .transpose()
        .round(3)
        .style.format(precision=3)
    )

In [38]:
D = {
    "poly": [
        ('food', 'UTS01', 473),	
        ('food_and_drink', 'UTS01', 203),	
        ('age_or_time', 'UTS01', 328),	
        ('clothing_and/or_undr', 'UTS01', 234),	
        ('family_relationships', 'UTS01', 487),	
        ('family_relationships', 'UTS01', 407),	
        ('locations_or_movemen', 'UTS01', 463),	
        ('transportation_or_mo', 'UTS01', 176),	
    ],
}
assert len(r[r.id_tup.isin(D['poly'])]) == len(D['poly'])

In [39]:
D_REVISED = {
    "poly": [
        ("food", "UTS01", 473),
        ("a number", "UTS01", 473),
        ("food and drink", "UTS01", 203),
        ("location", "UTS01", 203),
        ("age or time", "UTS01", 328),
        ("locations", "UTS01", 328),
        ("clothing", "UTS01", 234),
        ("people", "UTS01", 234),
        ("family relationships", "UTS01", 487),
        ("action or moevement", "UTS01", 487),
        ("family relationships", "UTS01", 407),
        ("location", "UTS01", 407),
        ("locations", "UTS01", 463),
        ("mother figure", "UTS01", 463),
        ('transportation', 'UTS01', 176),	
        ('color', 'UTS01', 176),	

    ],
}
json.dump(D_REVISED, open(f"polysemantic_{subject}.json", "w"), indent=0)  # , indent=4)

with pd.option_context(
    "display.max_rows", None, "display.max_columns", None, "display.max_colwidth", None
):
    rp = r[r.id_tup.isin(D["poly"])]
    display(
        rp[
            [
                "id",
                "top_explanation_init_strs",
                "top_explanation_init_strs_llama",
                # "explanation_init_strs",
                # "mean_score_synthetic",
                "fmri_test_corr",
                # "score_synthetic",
                "top_ngrams_module_25",
            ]
        ].transpose()
    )

,580,532,471,488,513,1259,1270,669
id,"('food', 'UTS01', 473),","('food_and_drink', 'UTS01', 203),","('age_or_time', 'UTS01', 328),","('clothing_and/or_undr', 'UTS01', 234),","('family_relationships', 'UTS01', 487),","('family_relationships', 'UTS01', 407),","('transportation_or_mo', 'UTS01', 176),","('locations_or_movemen', 'UTS01', 463),"
top_explanation_init_strs,food,food and drink,age or time,clothing and/or undressing,family relationships,family relationships,transportation or movement,locations or movement between locations
top_explanation_init_strs_llama,a number,location or place,"locations, such as corners, floors, houses, and corridors",people or crowds,action or movement,location,a reference to a color,presence of a mother figure
fmri_test_corr,0.326382,0.29798,0.294921,0.152566,0.246596,0.145463,0.203647,0.32077
top_ngrams_module_25,"[cucumber and mayonnaise, some sliced cucumber, sliced cucumber, and potatoes apples, sliced red onion, lemon slices, with lemon slices, cucumber bell peppers, can of apple, toothbrush toothpaste, bread wheat wheatgrass, wheat wheatgrass, bananas apples, hold a coconut, potatoes apples, shirt toothbrush toothpaste, c4 rice wheatgrass, mother put sausages, cut the apples, of seal meat, sauteed shiitake mushrooms, and the wasabi, piece of tomato, 're throwing napkins, little lemon juice]","[cucumber and mayonnaise, some sliced cucumber, sliced cucumber, steak and lobster, piece of tomato, and potatoes apples, can of apple, cucumber bell peppers, buffet of delicious, mother put sausages, sliced red onion, with lemon slices, had a ribeye, had steamed mussels, pizza and caesar, uh goat stew, eating chicken salad, bananas apples, some vegetable curry, a cheeseburger, roast beef, complimented the mussels, wore flannel, had mashed potato, salad and stale]","[i was twenty, was two am, by nine thirty, at twelve thirty, at four thirty, i was nineteen, about nine thirty, i was fifteen, eight hours south, in nineteen thirty, at age twenty, in nineteen fifty, in nineteen seventy, drove from vermont, i was fourteen, i was seventeen, in nineteen forty, i was eleven, age of nineteen, around twelve thirty, around six thirty, four hour drive, i was thirty, be six thirty, by twelve thirty]","[starts to undress, upside down naked, bath in bathrobes, lob rhetorical grenades, began to undress, in bathrobes, socks and loafers, in a bathrobe, have to undress, sat there naked, we caught eyes, danced throwing blankets, shaved her head, he locks eyes, and gesticulate wildly, daisy duke shorts, off his bathrobe, lifted her dress, his bathrobe, silver high heels, on a loincloth, shave her head, skirt heels, cigarette wrappers, rolled up sleeves]","[she recognized me, calls my brother, friends come over, she saw me, neighbor came over, he thought i, he imagined me, he thinks i, he saw me, mom my sister, see me nude, mom was crying, cousin my sister, he notices me, his pen pal, he knows i, his girlfriend he, seen me walking, heard him crying, brother thinking you, my former wife, her see him, he likes me, she sees my, friends saw me]","[hear boys laughing, walking in tandem, my mom sobbing, arms around me, 're laughing together, with my brother, laughing together, mom started crying, phone was ringing, with my sister, were holding hands, watched my son, helped my daughter, watched my daughter, mom was crying, laughing and reminiscing, walking towards us, we held hands, bunkmates start talking, we caught eyes, walking my dog, am asking dad, hugging each other, each other laughing, son comes running]","[later the doorbell, whole ride back, on the subway, driving home and, the following morning, on the flight, on the train, the next morning, on the bus, off the plane, the doorbell rang, very next morning, driving along and, went downstairs to, at work walking, at the airport, go downstairs to, walking home from, in the cafeteria, on the drive, came downstairs, on the trains, onto the bus, doorbell rang, downstairs to t

In [52]:
for i, k in enumerate(D_REVISED["poly"]):
    suff = "" if i % 2 == 0 else "_llama"
    print(
        str(k) + ":",
        '["' + '", "'.join(rp[rp.module_num == k[2]].iloc[0][f"top_ngrams_module_correct{suff}"][:6]) + '"]'
        + ",",
    )

('food', 'UTS01', 473): ["cucumber and mayonnaise", "some sliced cucumber", "sliced cucumber", "and potatoes apples", "sliced red onion", "lemon slices"],
('a number', 'UTS01', 473): ["thirty they said", "seven my mother", "thirty and i", "thirty minute", "fifteen he was", "six and i"],
('food and drink', 'UTS01', 203): ["cucumber and mayonnaise", "some sliced cucumber", "sliced cucumber", "steak and lobster", "piece of tomato", "and potatoes apples"],
('location', 'UTS01', 203): ["where was he", "where were we", "where home is", "where people were"],
('age or time', 'UTS01', 328): ["i was twenty", "was two am", "by nine thirty", "at twelve thirty", "at four thirty", "i was nineteen"],
('locations', 'UTS01', 328): ["floor looking for", "corners of", "corners for", "corner from", "corner of", "corner about"],
('clothing', 'UTS01', 234): ["starts to undress", "upside down naked", "bath in bathrobes", "began to undress", "in bathrobes", "socks and loafers"],
('people', 'UTS01', 234): ["cr

In [53]:
D_REVISED_NGRAMS = {
  ('food', 'UTS01', 473): ["cucumber and mayonnaise", "some sliced cucumber", "sliced cucumber", "and potatoes apples", "sliced red onion", "lemon slices"],
('a number', 'UTS01', 473): ["thirty they said", "seven my mother", "thirty and i", "thirty minute", "fifteen he was", "six and i"],
('food and drink', 'UTS01', 203): ["cucumber and mayonnaise", "some sliced cucumber", "sliced cucumber", "steak and lobster", "piece of tomato", "and potatoes apples"],
('location', 'UTS01', 203): ["where was he", "where were we", "where home is", "where people were"],
('age or time', 'UTS01', 328): ["i was twenty", "was two am", "by nine thirty", "at twelve thirty", "at four thirty", "i was nineteen"],
('locations', 'UTS01', 328): ["floor looking for", "corners of", "corners for", "corner from", "corner of", "corner about"],
('clothing', 'UTS01', 234): ["starts to undress", "upside down naked", "bath in bathrobes", "began to undress", "in bathrobes", "socks and loafers"],
('people', 'UTS01', 234): ["crowd and that", "crowd and by", "crowd which is", "crowd i was", "frat kids", "crowd full of"],
('family relationships', 'UTS01', 487): ["calls my brother", "mom my sister", "mom was crying", "cousin my sister", "brother thinking you", "my former wife"],
('action or moevement', 'UTS01', 487): ["fast forward to", "came to this", "came home late", "appeared out", "appeared beside me", "came to"],
('family relationships', 'UTS01', 407): ["my mom sobbing", "with my brother", "mom started crying", "with my sister", "watched my son", "helped my daughter"],
('location', 'UTS01', 407): ["where we were", "where we are", "where lisbon was", "where we lived", "where my grandparents", "along jake is"],
('locations', 'UTS01', 463): ["in his backyard", "into his backyard", "in her attic", "moved to brooklyn", "took the subway", "house in brooklyn"],
('mother figure', 'UTS01', 463): ["mother looked at", "unnerved by maternal", "mother carried me", "mother 's chest", "mother would take", "mother asked her"],
('transportation', 'UTS01', 176): ["whole ride back", "on the subway", "driving home and", "on the flight", "on the train", "on the bus"],
('color', 'UTS01', 176): ["blue eyes named", "rainbow", "orange i had", "white one yes", "red and blue", "blue thing just"],  
}

In [54]:
joblib.dump(D_REVISED_NGRAMS, f'polysemantic_ngrams_{subject}.pkl')

['polysemantic_ngrams_UTS01.pkl']